In [ ]:
# -*- coding: utf-8 -*-
"""test4i6.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/17BBGfZBN8F57KyoodixZpv1iKiAs-6NT
"""

#test2, with pressure loads
# Cantilever Beam under uniform pressure load
# C:\Users\shivm\OneDrive - Imperial College London\ME4\FYP\Abaqus python scripts

#----------------------------------------------------------------------------------------------------------------------

from abaqus import *
from abaqusConstants import *
import regionToolset


import numpy as np
from itertools import chain
import math


session.viewports['Viewport: 1'].setValues(displayedObject=None)

#----------------------------------------------------------------------------------------------------------------------
# # create the model
# mdb.models.changeKey(fromName='Model-1', toName='Cantilever Beam')
# beamModel = mdb.models['Cantilever Beam']
mdb.models.changeKey(fromName='Model-1', toName='Plate_model')
plateModel = mdb.models['Plate_model']




for j in range(0,6):
  # create the part

  import sketch
  import part

  #a) Sketch the beam cross section using rectangle tool
  plateProfileSketch = plateModel.ConstrainedSketch(name='Plate CS Profile',
                                              sheetSize=200)
  plateProfileSketch.rectangle(point1=(-100, 100), point2=(100, -100))

  plateProfileSketch.CircleByCenterPerimeter(center=(
    0.0, 0.0), point1=(-10.0, 0.0))
  

  plateProfileSketch.RadialDimension(curve=
    plateProfileSketch.geometry[6], radius=10+10*j, 
    textPoint=(-71.4530639648438, 0.294061660766602))

  # b) Create a 3d deformable part named "plate" by extruding the sketch
  platePart = plateModel.Part(dimensionality=TWO_D_PLANAR, name='Plate', type=
    DEFORMABLE_BODY)
  platePart.BaseShell(sketch=plateProfileSketch)

    
  

  
#   # # #----------------------------------------------------------------------------------------------------------------------
  # create material

  import material

  # Create material AISI 1005 steel by assigning mass density, youngs
  # modulus and poissons ratio
  plateMaterial = plateModel.Material(name='AISI 1005 Steel')
  plateMaterial.Density(table=((7872,),))
  plateMaterial.Elastic(table=((200E9, 0.29),))

  

#   # #----------------------------------------------------------------------------------------------------------------------
  # # create solid section and assign beam to it

  import section

  # create a section to assign to the beam
  PlateSection = plateModel.HomogeneousSolidSection(name='Plate Section', material='AISI 1005 Steel'
  , thickness=None)

  # assign the plate to this section
  platePart.Set(faces=platePart.faces.getSequenceFromMask((
    '[#1 ]', ), ), name='Set-4')
  platePart.SectionAssignment(offset=0.0, offsetField='', offsetType=MIDDLE_SURFACE, region=
    platePart.sets['Set-4'], sectionName='Plate Section', thicknessAssignment=FROM_SECTION)

  

#   # #----------------------------------------------------------------------------------------------------------------------
  # create the assembly

  import assembly

  # create part instance
  plateAssembly = plateModel.rootAssembly
  plateInstance = plateAssembly.Instance(name='Plate Instance', part=platePart, dependent=ON)

  

#   # #----------------------------------------------------------------------------------------------------------------------
  # create the step

  import step

  # create a static general step
  plateModel.StaticStep(name='Step-1', previous='Initial', description='Step-1')
 
  # plateAssembly.Set(cells=
  #   plateInstance.cells.getSequenceFromMask(
  #   ('[#1 ]', ), ), name='Set-1')

  
#   # #----------------------------------------------------------------------------------------------------------------------
  # create field output request

  # change the name of field output request 'F-Output-1' to 'Selected Field Outputs'
  plateModel.fieldOutputRequests.changeKey(fromName='F-Output-1', toName='Selected Field Outputs')

  # since F-Output-1 is applied at the 'Apply Load' step by default, 'Selected Field Outputs' will be too
  # choose which results to output
  plateModel.fieldOutputRequests['Selected Field Outputs'].setValues(variables=('S', 'E', 'PEMAG', 'U', 'RF', 'CF'))

  

#   # #----------------------------------------------------------------------------------------------------------------------
  # create the history output request (I don't think I used this section,  just copied it)

  # we try a slightly different method from that used in the field output request
  # create a new history output request called 'Default History Outputs' and assign both the step and the variables
  plateModel.HistoryOutputRequest(name='Default History Outputs', createStepName='Step-1', variables=PRESELECT)

  # now delete the original history output  request 'H-Output-1'
  del plateModel.historyOutputRequests['H-Output-1']

  

#   # # #----------------------------------------------------------------------------------------------------------------------

  v1 = plateInstance.vertices
  verts1 = v1.getSequenceFromMask(mask=('[#8 ]', ), )
  region = plateAssembly.Set(vertices=verts1, name='Set-1')

  plateModel.DisplacementBC(name='BC-1', createStepName='Initial', 
      region=region, u1=SET, u2=UNSET, ur3=SET, amplitude=UNSET, 
      distributionType=UNIFORM, fieldName='', localCsys=None)

  verts1 = v1.getSequenceFromMask(mask=('[#10 ]', ), )
  region = plateAssembly.Set(vertices=verts1, name='Set-2')

  session.viewports['Viewport: 1'].view.setValues(nearPlane=253.793, 
      farPlane=311.893, width=205.342, height=94.2157, cameraPosition=(19.9893, 
      -7.01929, 282.843), cameraTarget=(19.9893, -7.01929, 0))

  session.viewports['Viewport: 1'].view.setValues(nearPlane=253.825, 
      farPlane=311.861, width=205.368, height=94.2276, cameraPosition=(20.6716, 
      -7.01929, 282.843), cameraTarget=(20.6716, -7.01929, 0))
  session.viewports['Viewport: 1'].view.setValues(nearPlane=253.825, 
      farPlane=311.861, cameraUpVector=(0.00070316, 1, 0))


  e1 = plateInstance.edges
  edges1 = e1.getSequenceFromMask(mask=('[#8 ]', ), )
  region = plateAssembly.Set(edges=edges1, name='Set-3')
  plateModel.DisplacementBC(name='BC-3', createStepName='Initial', 
      region=region, u1=UNSET, u2=SET, ur3=UNSET, amplitude=UNSET, 
      distributionType=UNIFORM, fieldName='', localCsys=None)
  session.viewports['Viewport: 1'].view.setValues(nearPlane=253.662, 
      farPlane=312.024, width=218.337, height=100.178, viewOffsetX=5.85374, 
      viewOffsetY=0.0599978)
  session.viewports['Viewport: 1'].assemblyDisplay.setValues(step='Step-1')
  session.viewports['Viewport: 1'].view.setValues(nearPlane=244.45, 
      farPlane=321.236, width=286.696, height=131.543, viewOffsetX=-6.23524, 
      viewOffsetY=-10.4826)
#    # #----------------------------------------------------------------------------------------------------------------------

#     #pressure load
  s1 = plateInstance.edges
  side1Edges1 = s1.getSequenceFromMask(mask=('[#2 ]', ), )
  region = plateAssembly.Surface(side1Edges=side1Edges1, name='Surf-1')
  plateModel.Pressure(name='Load-1', createStepName='Step-1', 
      region=region, distributionType=UNIFORM, field='', magnitude=-1.0, 
      amplitude=UNSET)


  

  
#   #----------------------------------------------------------------------------------------------------------------------
  # create the mesh

  import mesh
  platePart.setElementType(elemTypes=(mesh.ElemType(
      elemCode=CPS4R, elemLibrary=STANDARD, secondOrderAccuracy=OFF, 
      hourglassControl=DEFAULT, distortionControl=DEFAULT), mesh.ElemType(
      elemCode=CPS3, elemLibrary=STANDARD)), regions=(
      platePart.faces.getSequenceFromMask((
      '[#1 ]', ), ), ))
  platePart.seedPart(deviationFactor=0.1, 
      minSizeFactor=0.1, size=1.0)
  platePart.generateMesh()
  platePart.deleteMesh(regions=
      platePart.faces.getSequenceFromMask((
      '[#1 ]', ), ))
  platePart.seedEdgeBySize(constraint=FINER, 
      deviationFactor=0.1, edges=
      platePart.edges.getSequenceFromMask((
      '[#1 ]', ), ), minSizeFactor=0.1, size=0.5)
  platePart.generateMesh()

  
  
  

  

  
 # #----------------------------------------------------------------------------------------------------------------------
  # # # create and run the job

  import job

  # create the job
  # the job number is incremented with every loop
  mdb.Job(name='PlateJob', model='Plate_model', type=ANALYSIS, explicitPrecision=SINGLE,
          nodalOutputPrecision=SINGLE, description='Job simulates a loaded plate',
          parallelizationMethodExplicit=DOMAIN, multiprocessingMode=DEFAULT, numDomains=1, userSubroutine='',
          numCpus=1, memory=50, memoryUnits=PERCENTAGE, scratch='', echoPrint=OFF, modelPrint=OFF,
          contactPrint=OFF, historyPrint=OFF)
  
  mdb.Job(name='PlateJob', model='Plate_model', type=ANALYSIS, explicitPrecision=SINGLE,
    description='Job simulates a loaded plate',
    atTime=None, contactPrint=OFF, echoPrint=OFF,
    getMemoryFromAnalysis=True, historyPrint=OFF, 
    memory=90, memoryUnits=PERCENTAGE, modelPrint=OFF, 
    multiprocessingMode=DEFAULT, nodalOutputPrecision=SINGLE, 
    numCpus=1, numGPUs=0, queue=None, resultsFormat=ODB, scratch='',
    userSubroutine='', waitHours=0, waitMinutes=0)

  # run the job
  mdb.jobs['PlateJob'].submit(consistencyChecking=OFF)

  # Do not return control till job is finished running
  mdb.jobs['PlateJob'].waitForCompletion()

  


  # # end of run job
  # # ----------------------------------------------------------------------------------------------------------------------
  # # post processing

  import visualization

  # the output database is named
  beam_viewport = session.Viewport(name='Beam Results Viewport')
  beam_Odb_Path = 'PlateJob.odb'
  an_odb_object = session.openOdb(name=beam_Odb_Path)
  beam_viewport.setValues(displayedObject=an_odb_object)
  beam_viewport.odbDisplay.display.setValues(plotState=(DEFORMED, ))

  session.viewports['Viewport: 1'].setValues(displayedObject=an_odb_object)
  # set report name and format for each job
  report_name_and_path = 'Report'+str(j)+'.csv'
  #report_name_and_path = 'Report'+'.csv'

  # writes the report with the variables you choose. I've chosen sigma_33 at every integration point
  session.fieldReportOptions.setValues(reportFormat=COMMA_SEPARATED_VALUES)
  session.writeFieldReport(fileName=report_name_and_path, append=OFF,
      sortItem='Element Label', odb=an_odb_object, step=0, frame=1,
      outputPosition=INTEGRATION_POINT, variable=(('S', INTEGRATION_POINT, ((
      COMPONENT, 'S22'), )), ))
  mdb.saveAs(pathName='C:\Abaqus_sims\Plate\SCcripts\Script1\Test11\Testcae')


  


  
  
